In [22]:
import re, numpy as np, pandas as pd, time
from collections import Counter
from extended_buckwalter import transliterate, remove_diacritics
from scipy.spatial.distance import hamming
from sklearn.metrics import accuracy_score

ModuleNotFoundError: No module named 'sklearn.metrics'

In [ ]:
pick = '2'

f = open('properties.txt').read()

prefix, suffix, labels = [], [], []
for x, i in enumerate(f.splitlines()[3:]):
    i = re.sub('[ ][ ]+', ' ', i)
    locs, tags, mtag, word, form, rems, prfx, sffx = i.split(' | ')

    if not locs.startswith(pick + ':'): continue

    prfx = ''.join(prfx.split())
    sffx = ''.join(sffx.split())

    prefix.append(prfx if prfx != '-' else None)
    suffix.append(sffx if sffx != '-' else None)
    labels.append(tags.split()[0])
labels = np.asarray(labels)

trans = open('transliterasi.txt').read()

In [ ]:
I = 0
data = []
for sents in trans.splitlines():

    if not sents.startswith(pick + ' |'): continue

    sents = sents.split(' | ')[-1].split()
    lensents = len(sents)

    for i, word in enumerate(sents):

        fwpos = [word, labels[I]]

        fword, fpost = [], []
        for v in range(-3, 4):
            if i + v >= 0 and i + v < lensents:
                fword.append(sents[i + v])
                fpost.append(labels[I + v])
            else:
                fword.append('-')
                fpost.append('-')

        fprfx, temp = [], []
        for v in range(1, 5):
            if i + v < lensents:
                if prefix[I + v] != None:
                    temp.append(prefix[I + v])
                else:
                    temp.append('')
            else:
                temp.append('')
            vals = ''.join(temp)
            fprfx.append(vals if vals != '' else '-')

        fsffx, temp = [], []
        for v in range(0, -4, -1):
            if i + v >= 0:
                if suffix[I + v] != None:
                    temp.append(suffix[I + v])
                else:
                    temp.append('')
            else:
                temp.append('')
            vals = ''.join(temp)
            fsffx.append(vals if vals != '' else '-')

        '''
        fbins = [True if word.isupper() else False]
        fbins.append(True if word.islower() else False)

        x = re.findall('\d+',word)
        fbins.append(True if x != [] else False)
        '''

        fbins = ['False', 'False']
        x = re.findall('\d+', word)
        fbins.append('True' if x != [] else 'False')

        flens = [str(len(remove_diacritics(word)))]

        feature = fwpos + fword + fpost + fprfx + fsffx + fbins + flens
        data.append(feature)

        I += 1


In [ ]:
data = np.asarray(data)

arab = np.copy(data)
for i in range(len(data)):
    for j in range(len(data[0])):
        if j in [0, 2, 3, 4, 5, 6, 7, 8, 16, 17, 18, 19, 20, 21, 22, 23]:
            if data[i][j] in ['-', 'none']:
                continue
            trans = data[i][j].replace('none', '؟؟؟؟')
            trans = transliterate(trans)
            trans = trans.replace('؟؟؟؟', 'none')
            arab[i, j] = trans

In [ ]:
cols = """
WORD POS
W-3 W-2 W-1 W0 W1 W2 W3
P-3 P-2 P-1 P0 P1 P2 P3
S1 S1,S2 S1,S2,S3 S1,S2,S3,S4
S0 S0,S-1 S0,S-1,S-2 S0,S-1,S-2,S-3
uppercase lowercase contain_number
length
""".split()

writer = pd.ExcelWriter('feature.xlsx')

df = pd.DataFrame(data)
df.columns = cols
df.to_excel(writer, sheet_name='transliterasi')

df = pd.DataFrame(arab)
df.columns = cols
df.to_excel(writer, sheet_name='arabic')
writer.save()

data_ori = np.copy(data)
data = np.delete(data, [3, 4, 5, 6, 7, 8, 9], 1)

In [ ]:
cols = """
W-3 W-2 W-1 W0 W1 W2 W3
P-3 P-2 P-1 P0 P1 P2 P3
S1 S1,S2 S1,S2,S3 S1,S2,S3,S4
S0 S0,S-1 S0,S-1,S-2 S0,S-1,S-2,S-3
length
""".split()

Freq = Counter(labels)

data_train, label_train = [], []
data_test, label_test = [], []

data_train_ori, data_test_ori = [], []

Itrain, Itest = [], []

for i in sorted(set(labels)):
    n = Freq[i]
    m = int(np.ceil(n / 2))

    index_clas = np.arange(len(data))[labels == i]

    index_train = np.sort(np.random.choice(index_clas, m, replace=False))
    index_test = np.asarray([j for j in index_clas if j not in index_train])

    for j in index_train:
        data_train.append(data[j])
        data_train_ori.append(data_ori[j])
        label_train.append(labels[j])
    for j in index_test:
        data_test.append(data[j])
        data_test_ori.append(data_ori[j])
        label_test.append(labels[j])

    Itrain += index_train.tolist()
    Itest += index_test.tolist()

data_train = np.asarray(data_train)
data_test = np.asarray(data_test)
label_train = np.asarray(label_train)
label_test = np.asarray(label_test)

data_train_ori = np.asarray(data_train_ori)
data_test_ori = np.asarray(data_test_ori)

writer = pd.ExcelWriter('data_train_test.xlsx')

df = pd.DataFrame(data_train)
df.columns = cols
df.index = Itrain
df.to_excel(writer, sheet_name='data train')

df = pd.DataFrame(data_test)
df.columns = cols
df.index = Itest
df.to_excel(writer, sheet_name='data test')
writer.save()

In [ ]:
def k_nearest_neighbors(data_train, label_train, data_test):
    K = 7
    N = len(data_test)

    label_predict = []
    for i, data_i in enumerate(data_test):

        dist = []
        for data_j in data_train:
            dist.append(hamming(data_i, data_j))

        sort = np.argsort(dist)

        I = 0
        while True:

            post = label_train[sort[:K + I]]
            freq = Counter(post)

            tags, maxf = freq.most_common(1)[0]

            if list(freq.values()).count(maxf) == 1: break
            I += 1

        label_predict.append(tags)

    return label_predict


In [ ]:
def naive_bayes(data_train, label_train, data_test):
    N = len(data_train)
    V = len(data_train[0])

    clas = sorted(set(label_train))
    Freq = Counter(label_train)

    P_label = {}
    for pos in clas:
        P_label[pos] = Freq[pos] / N

    P_xiyj = {}
    for i in range(V):
        cols = data_train[:, i]
        setcols = sorted(set(cols))
        for j, word in enumerate(setcols):

            freq = Counter(label_train[cols == word])

            for pos in clas:
                key = word + '|' + str(i) + '|' + pos
                P_xiyj[key] = freq[pos] / Freq[pos]

    label_predict = []
    for data_i in data_test:

        P_i = []
        for pos in clas:

            P_ij = P_label[pos]
            for i, word in enumerate(data_i):

                key = word + '|' + str(i) + '|' + pos

                try:
                    P_xy = P_xiyj[key]
                except KeyError:
                    continue

                P_ij = P_ij * P_xy

            P_i.append(P_ij)

        label_predict.append(clas[np.argmax(P_i)])
    return label_predict

In [ ]:
writer = pd.ExcelWriter('result.xlsx')

print('>> NAIVE BAYES: ')
start = time.time()
predict_nb = naive_bayes(data_train, label_train, data_test)
accur = accuracy_score(predict_nb, label_test)
print('   ACCURACY:', str(np.round(accur * 100, 2)) + '%')
print('   RUNTIME :', time.time() - start, 'detik\n')

df = pd.DataFrame(np.asarray([list(i[:2]) + [j] for i, j in zip(data_test_ori, predict_nb)]))
df.columns = ['WORD', 'POS_ASLI', 'POS_HASIL']
df.index = Itest
df.to_excel(writer, sheet_name='naive bayes')

print('>> K-NEAREST NEIGHBORS:')
start = time.time()
predict_knn = k_nearest_neighbors(data_train, label_train, data_test)
accur = accuracy_score(predict_knn, label_test)
print('   ACCURACY:', str(np.round(accur * 100, 2)) + '%')
print('   RUNTIME :', time.time() - start, 'detik\n')

df = pd.DataFrame(np.asarray([list(i[:2]) + [j] for i, j in zip(data_test_ori, predict_knn)]))
df.columns = ['WORD', 'POS_ASLI', 'POS_HASIL']
df.index = Itest
df.to_excel(writer, sheet_name='knn')
writer.save()
